In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

data_transforms = {
    'train' : transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    'val' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
}

In [2]:
import os
from torchvision import datasets

data_dir = 'ants_bees'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],batch_size=4,shuffle=True,num_workers=4) for x in ['train','val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
import time
import copy

def train_model(model, criterion, optimizer, num_epochs = 10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-'*10)

        for phase in ['train','val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _,preds = torch.max(outputs,1)
                loss = criterion(outputs,labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item()*inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss/dataset_sizes[phase]
        epoch_acc = running_corrects.double()/dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
    print()
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model

In [5]:
import torchvision.models as models

model_ft = models.resnet18(pretrained=False, num_classes=2)
model_ft = model_ft.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model_ft.parameters(),lr=0.001,momentum=0.9)

model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=10)

/Users/daejin/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/daejin/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 0/9
----------
val Loss: 0.7396 Acc: 0.4444
Epoch 1/9
----------
val Loss: 0.7396 Acc: 0.4444
Epoch 2/9
----------
val Loss: 0.7396 Acc: 0.4444
Epoch 3/9
----------
val Loss: 0.7396 Acc: 0.4444
Epoch 4/9
----------
val Loss: 0.7396 Acc: 0.4444
Epoch 5/9
----------
val Loss: 0.7396 Acc: 0.4444
Epoch 6/9
----------
val Loss: 0.7396 Acc: 0.4444
Epoch 7/9
----------
val Loss: 0.7396 Acc: 0.4444
Epoch 8/9
----------
val Loss: 0.7396 Acc: 0.4444
Epoch 9/9
----------
val Loss: 0.7396 Acc: 0.4444

Training complete in 4m 7s
Best val Acc: 0.444444
